# Emcee GP Study

An emcee sampler notebook, testing various things about the emcee package.

In [1]:
%matplotlib notebook 
#restart the kernel if switching from inline to notebook
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import corner

import numpy as np
import numpy.linalg
import scipy.stats
from scipy.signal import argrelextrema
import pandas as pd

import emcee
import george
from emcee import PTSampler

from subsample import subsample # daniela's code
from emcee_utils import walker_params, plot_gpfit
from plotting import plot_lightcurve, plot_folded_lightcurve, plot_mcmc_sampling_results, plot_steps

In [2]:
def prior(params):

    """
    Calculated the log of the prior values, given parameter values.

    Parameters
    ----------
    params : list
        List of all kernel parameters

    param[0] : float
        mean (between 0 and 2)

    param[1] : float
        log amplitude (between -10 and 10)

    param[2] : float
        gamma (log gamma between 0.1 and )

    param[3] : float
        log period (period between 1h and 24hrs)

    Returns
    -------
    sum_log_prior : int
        sum of all log priors (-inf if a parameter is out of range)

    """

    p_mean = scipy.stats.uniform(0,20).logpdf(params[0])
    p_log_amp = scipy.stats.uniform(-10,30).logpdf(params[1])
    p_log_gamma = scipy.stats.uniform(np.log(0.1), (np.log(40)-np.log(0.1))).logpdf(np.log(params[2]))
    p_period = scipy.stats.uniform(np.log(0.5/24), -np.log(0.5/24)).logpdf((params[3]))

    sum_log_prior =  p_mean + p_log_amp + p_log_gamma + p_period

    if np.isnan(sum_log_prior) == True:
        return -np.inf

    return sum_log_prior

def logl(params, gp, tsample, fsample, flux_err):
     # compute lnlikelihood based on given parameters
     gp.set_parameter_vector(params)
     gp.compute(tsample, flux_err)

     lnlike = gp.lnlikelihood(fsample)

     return lnlike

def post_lnlikelihood(params):

    """
    Calculates the posterior likelihood from the log prior and
    log likelihood.

    Parameters
    ----------
    params : list
        List of all kernel parameters

    Returns
    -------
    ln_likelihood : float
        The posterior, unless the posterior is infinite, in which case,
        -1e25 will be returned instead.

    """
    # calculate the log_prior
    log_prior = prior(params)

    # return -inf if parameters are outside the priors
    if np.isneginf(log_prior) == True:
        return -np.inf

    # compute lnlikelihood based on given parameters
    lnlike = logl(params, gp, tsample, fsample, flux_err)

    try:
        gp.compute(tsample, flux_err)
        ln_likelihood = gp.lnlikelihood(fsample)+log_prior
    except np.linalg.LinAlgError:        
        ln_likelihood = -1e25

    return ln_likelihood if np.isfinite(ln_likelihood) else -1e25

First we have to extract the time stamps and flux from any data files we might have. For simulated data, we have to sample it at a candence that would be similar to real observations, meaning we have 3 types of flux and time data.
1. __data.time & data.flux__ : all the time and flux data from the simulated data. This might span multiple days-months and has a data point for every 30 seconds using DAMIT generated data

2. __time & flux__ : the time and flux data for a smaller range of dates than all_time and all_flux. This is essentially the observing window in which we are working with and time is what we will be projecting our gp fits onto

3. __tsample & fsample__ : the time and flux data sampled from the time and flux data. The sampling of this is more realistic (every 10 minutes instead of every 0.5 minutes)

In [3]:
asteroid = '1291'

true_log_p = {'3200':-1.896021, '1291':-1.45813055, 
              '221':-0.8321219, '1388':-0.69789175}
true_p = {'3200':3.603957, '1291':5.58410, 
              '221':10.443, '1388':11.9432}

txt = '../data/'+str(asteroid)+'_lc_49627_to_49787.txt'

data = pd.read_csv(txt, delimiter=' ',
                 header=None, names=['time','flux'], dtype={'time':float, 'flux':float})

days, delay = 5, 50

# convert days to points
span = 2880 * days
start_pt = 2880 * delay

time = np.array(data.time[start_pt:span+start_pt])
flux = np.array(data.flux[start_pt:span+start_pt])

flux_err = np.ones_like(flux) * np.std(flux)/10.0
tsample, fsample, flux_err = subsample(time, flux, flux_err=flux_err, npoints=100, kind="telescope")

fig, ax = plt.subplots(1,1, figsize=(8,4))
#ax.plot(time, flux, '-', alpha=0.5, label="Original : " + str(round(true_log_p[asteroid], 5)))
ax.set_title("%i nights, %i data points"%(days, len(fsample)))
ax.set_xlabel("Days (JD)")
ax.errorbar(tsample, fsample, yerr=flux_err, fmt="o", markersize=5,
            color="black", zorder=10, label="Sample : " + str(len(tsample)))
ax.legend()

In [4]:
data  = pd.read_csv("../data/221_lc_49627_to_49787.txt")

Let's see what a Lomb-Scargle periodogram predicts the period should be based on our data so far.

In [6]:
from lombscargle import make_lsp
from astropy.stats import LombScargle

freq, power = make_lsp(tsample, fsample, flux_err, p_max=5.0)

best_freq = freq[np.argmax(power)]
best_period = 1./best_freq
best_log_period = np.log(1./best_freq)

fig, (bx,cx,dx) = plt.subplots(1,3, figsize=(9,2.5))
fig.set_tight_layout('tight')
bx.plot(freq, power)
bx.set_xlabel('Frequency')
bx.set_ylabel('Power')
bx.vlines(best_freq, 0, 1, colors='orange', linestyles='--', 
          label = 'Best freq : ' + str(round(best_freq, 5)))
bx.legend()

cx.plot((1./freq),power)
cx.set_xlabel('Period')
cx.vlines(best_period, 0, 1, colors='orange', linestyles='--', 
          label = 'Best period : ' + str(round(1./best_freq, 5)))
cx.set_xlim([0,1])
cx.legend()

dx.plot(np.log(1./freq),power)
dx.set_xlabel('Log Period')
dx.vlines(np.log(1./best_freq), 0, 1, colors='orange', linestyles='--', 
          label = 'Best log period : ' + str(round(np.log(1./best_freq), 5)))
dx.set_xlim([-3.3,0])
dx.legend()

y_fit = LombScargle(tsample, fsample, flux_err).model(time, best_freq)
#ax.plot(time, y_fit, label = "Lomb-Scargle fit : " + str(round(best_log_period, 5)))
#ax.legend()

In [7]:
# determine the indices of local power maxima  
best_idx = argrelextrema(power, np.greater)

# sort these indices based on actual power value
# reverse list so max is read first
indices = np.argsort(power[best_idx[0]])[::-1]

# sort our original indices based on the new 
# power-sorted indices
best_idx = (best_idx[0]).T[indices]
best_freqs = freq[best_idx].T

In [8]:
new_freq = best_freqs[0]
new_period = 1./new_freq
new_log_period = np.log(1./new_freq)
y_fit = LombScargle(tsample, fsample, flux_err).model(time, new_freq)
#ax.plot(time, y_fit, '--', label = "Lomb-Scargle fit : " + str(round(new_log_period, 5)))
#ax.legend()

bx.vlines(new_freq, 0, 1, linestyles='--', alpha=0.5,
          label = 'New fit : ' + str(round(new_freq, 5)))
bx.legend()

cx.vlines(1./new_freq, 0, 1, linestyles='--',  alpha=0.5,
          label = 'New period : ' + str(round(1./new_freq, 5)))
cx.legend()
dx.vlines(new_log_period, 0, 1, linestyles='--',  alpha=0.5,
          label = 'New log period : ' + str(round(new_log_period, 5)))
dx.legend()

In [9]:
plot_folded_lightcurve(tsample, fsample, new_period)# , true_lightcurve=[time,flux])

How many walkers do we want? So far there are 4 parameters/dimensions we want to study: mean, log_amp, gamma, and log_period.

In [10]:
ndim, nwalkers = 4, 100

In [11]:
# initialize walker parameters
best_log_amp = np.log(fsample.max()-fsample.min())
params = [np.mean(fsample), best_log_amp, 1, best_log_period]
p0, gp = walker_params(params, fsample, flux_err, nwalkers, cov_scale=1)
#plot_gpfit(time, fsample, flux_err, gp, ax)

amp : 0.6192530987072306
params : [ mean, log_amp, gamma, log_period]
params : [1.0058487945897792, -0.47924120665711273, 1, -2.1487430075219596]
cov matrix : 
[[0.01005849 0.         0.         0.        ]
 [0.         0.00479241 0.         0.        ]
 [0.         0.         0.01       0.        ]
 [0.         0.         0.         0.02148743]]


In [12]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, post_lnlikelihood, threads=1)

Note : If your sampler fails within the first couple of seconds, try making the cov_scale smaller.

In [13]:
%%time
mcmc_sampling = sampler.run_mcmc(p0, 50)

/home/christina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log


CPU times: user 12.7 s, sys: 11.7 ms, total: 12.7 s
Wall time: 12.7 s


In [15]:
def plot_steps(sampler, dims=None, p0=None, data_pts=None):
    fig, ax = plt.subplots(2, 2, figsize=(7,6))
    fig.subplots_adjust(wspace=0.25, hspace=0.3)

    fig.suptitle("Data points: " + str(data_pts) + "\nMean acceptance fraction: {0:.3f}".format(np.mean(sampler.acceptance_fraction)))

    axs = [ax[0,0], ax[0,1], ax[1,0], ax[1,1]]

    x = np.arange(sampler.iterations)

    for i in range(sampler.dim):
        axs[i].set_xlabel('Step Number')
        axs[i].set_ylabel('{}'.format(dims[i]))

        for j in range(len(sampler.chain)):
            param = sampler.chain[j,:,i]
            axs[i].plot(x, param, 'k-', alpha=0.3)
            # fit might guess period is time range of sampling

        flatchain = sampler.flatchain[:,i]


    return axs

In [ ]:
plot_steps(sampler, dims = ['mean', 'log_amp', 'gamma', 'log_period'], p0=[params], data_pts=len(fsample))

In [ ]:
fig, fx = plt.subplots(1,1)

x = np.arange(sampler.iterations)
for i in np.arange(100):
    fx.plot(x, sampler.lnprobability[i,:], 'k', alpha=0.3)
fx.set_xlabel("Steps")
fx.set_ylabel("Ln Probability")

In [ ]:
# sampler.chain[chain, step, dimension]
end_period = sampler.chain[:,-1,-1]
end_period.sort()

fig, fx = plt.subplots(1,1)
fx.hist(end_period)
fx.set_xlabel('log_period')

In [ ]:
save_chain('test_chain2', sampler)

In [ ]:
new_data = read_chain('test_chain2', (100,50,4))

In [ ]:
def save_chain(file_name, sampler):
    header = str(sampler.chain.shape)
    np.savetxt(file_name, sampler.flatchain, header=header)
    return 

In [ ]:
def read_chain(file_name, shape):
    new_data = np.genfromtxt(file_name, delimiter=' ')
    # data shape can be found as the first commented line in the txt file
    new_data = new_data.reshape(shape)
    return new_data

Now we can save our data for later.

Or read it out if we want to compare. If you want to plot the saved data, make sure to indicate it when plotting by setting __from_saved__ to True.

In [ ]:
test = read_chain('test', (100,100,4))

In [ ]:
plot_steps(test, dims = ['mean', 'log_amp', 'gamma', 'log_period'], p0=p0, data_pts=len(fsample), from_saved=True)

In [ ]:
cgp.plot_hist(sampler)

In [ ]:
plot_mcmc_sampling_results(tsample, fsample, flux_err, gp, sampler, namestr='test', 
                           true_lightcurve = [time, flux], true_period=true_p[asteroid])

# Misc stuff down here


## Skip the following cell if you are working with simulated data.

Here's an alternative way of doing things if you're working with real data measurements. Since we're working with real data, we are essentially given __tsample__ and __fsample__. So we don't need to sample any points but this also means we don't know what the real light curve looks like. We can still generate a __time__ since it just needs to span from the first observation of __tsample__ to the last.

In [ ]:
txt = '../data/asteroid_csv/2018LF05.csv'
data = pd.read_csv(txt)

tsample = data.jd
fsample = data.magpsf
flux_err = data.sigmapsf
data_pts = len(tsample)

tsample.iloc[-1]

time = pd.Series(np.linspace(tsample.iloc[0], tsample.iloc[-1], 1000))

fig, ax = plt.subplots(1,1, figsize=(8,4))
ax.errorbar(tsample, fsample, yerr=flux_err, fmt = 'k.', label="Sample : " + str(len(tsample)))
ax.legend()

## Failed Hack Day Experiment

In [ ]:
from bokeh.plotting import figure, show, gridplot, ColumnDataSource
from bokeh.io import output_notebook
output_notebook()

data = sampler.chain.T.reshape(4*sampler.iterations,len(sampler.chain))
df = pd.DataFrame(
    data=data,
    index=pd.MultiIndex.from_product([['mean', 'log_amp','gamma','log_p'], np.arange((sampler.iterations))], names=['parameter','steps']),
    columns=np.arange(len(sampler.chain)))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,tap,box_select"

x_list_of_list = []
for i in range(0,len(df.columns)):
    x_list_of_list.append(df.index.levels[1])

mean_list_of_list = df.xs('mean', level=0).values.T.tolist()
p1 = figure(tools=TOOLS, width=350, plot_height=350, title="Trace Plot")
#p1.multi_line(x_list_of_list, mean_list_of_list)

log_amp_list_of_list = df.xs('log_amp', level=0).values.T.tolist()
p2 = figure(tools=TOOLS, width=350, plot_height=350, title="Trace Plot")
#p2.multi_line(x_list_of_list, log_amp_list_of_list)

source = ColumnDataSource(df)

for j in df.columns:
    # need to repmat the name to be same dimension as index
    name_for_display = np.tile(j, len(df.index.levels[1]))
    source = ColumnDataSource({'x': df.index.levels[1].values, 'mean': df.xs('mean', level=0)[j].values,
                               'log_amp': df.xs('log_amp', level=0)[j].values, 'chain': name_for_display})
    
    p1.circle('x', 'mean', source = source)
    p2.circle('x', 'log_amp', source = source)
    

p = gridplot([[p1, p2]])
show(p)

In [13]:
datadir = "../data/"
filename = "phaethon_damit.txt"
data  = pd.read_csv(datadir+filename, header=None, delim_whitespace=True)

In [14]:
tsample = data[0]
tsample

0      2.453344e+06
1      2.453344e+06
2      2.453344e+06
3      2.453344e+06
4      2.453344e+06
5      2.453344e+06
6      2.453344e+06
7      2.453344e+06
8      2.453344e+06
9      2.453344e+06
10     2.453344e+06
11     2.453344e+06
12     2.453344e+06
13     2.453344e+06
14     2.453344e+06
15     2.453344e+06
16     2.453344e+06
17     2.453344e+06
18     2.453344e+06
19     2.453344e+06
20     2.453344e+06
21     2.453344e+06
22     2.453344e+06
23     2.453344e+06
24     2.453344e+06
25     2.453344e+06
26     2.453344e+06
27     2.453344e+06
28     2.453344e+06
29     2.453344e+06
           ...     
112    2.453345e+06
113    2.453345e+06
114    2.453345e+06
115    2.453345e+06
116    2.453345e+06
117    2.453345e+06
118    2.453345e+06
119    2.453345e+06
120    2.453345e+06
121    2.453345e+06
122    2.453345e+06
123    2.453345e+06
124    2.453345e+06
125    2.453345e+06
126    2.453345e+06
127    2.453345e+06
128    2.453345e+06
129    2.453345e+06
130    2.453345e+06
